In [ ]:
from pathlib import Path
from ebmdatalab import bq

from utils import read_sql_query
from config import DATA_DIR

PROJECT_ID = "ebmdatalab"
CREDENTIALS_PATH = 'notebooks/cred/bq-service-account.json'


In [ ]:
query = read_sql_query("andexanet.sql")
df = bq.cached_read(sql=query, csv_path=Path(DATA_DIR, "andexanet.csv"), use_cache=False)
 

In [ ]:
df.head()

In [ ]:
all_ods_codes_query = """
WITH latest_six_months AS (
    SELECT DISTINCT period
    FROM scmd.org_ae_status
    WHERE period >= TIMESTAMP(DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH))
    ORDER BY period DESC
    LIMIT 6
),
ae_status_last_six_months AS (
    SELECT 
        ods_code,
        MAX(CAST(has_ae AS INT64)) AS has_ae_last_six_months
    FROM scmd.org_ae_status
    WHERE period IN (SELECT period FROM latest_six_months)
    GROUP BY ods_code
)
SELECT DISTINCT 
    CASE
        WHEN ods.successor_ods_code IS NOT NULL AND ods.successor_ods_code != 'None'
        THEN ods.successor_ods_code
        ELSE ods.ods_code
    END AS ods_code,
    CASE
        WHEN ods.successor_ods_code IS NOT NULL AND ods.successor_ods_code != 'None'
        THEN successor_org.ods_name
        ELSE ods.ods_name
    END AS ods_name,
    CASE
        WHEN ae.ods_code IS NULL THEN NULL
        WHEN ae.has_ae_last_six_months = 1 THEN TRUE
        ELSE FALSE
    END AS has_ae_last_six_months
FROM 
    scmd.ods_mapped AS ods
LEFT JOIN 
    scmd.ods_mapped AS successor_org
ON
    ods.successor_ods_code = successor_org.ods_code
LEFT JOIN
    ae_status_last_six_months AS ae
ON 
    CASE
        WHEN ods.successor_ods_code IS NOT NULL AND ods.successor_ods_code != 'None'
        THEN ods.successor_ods_code
        ELSE ods.ods_code
    END = ae.ods_code
WHERE
    (ods.successor_ods_code IS NOT NULL AND ods.successor_ods_code != 'None')
    OR (
        (ods.operational_closed_date IS NULL OR ods.operational_closed_date >= '2019-01-01')
        AND (ods.legal_closed_date IS NULL OR ods.legal_closed_date >= '2019-01-01')
    )
"""

all_ods_codes = bq.cached_read(sql=all_ods_codes_query, csv_path=Path(DATA_DIR, "all_ods_codes.csv"), use_cache=False)
all_ods_codes.to_csv(Path(DATA_DIR, "all_ods_codes.csv"), index=False)
all_ods_codes.head()